In [10]:
from sqlalchemy import create_engine
import pandas as pd
import unicodedata
import json
# # Imports the methods needed to abstract classes into tables
# from sqlalchemy.ext.declarative import declarative_base

# # Allow us to declare column types
# from sqlalchemy import Column, Integer, String, Float 

In [11]:

rds_connection_string = "postgres:PostgreSQL@localhost:5432/trfmk_scrape_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [12]:
engine.table_names()

['leagues', 'teams', 'players']

In [21]:
pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING(team_id)) USING (league_id)', con=engine).head()

,league_id,country,tier,league_name,league_link,team_id,club,squad,foreigners,avg_market_value_m,...,position,age,nat,height,foot,dt_joined,prev_team,contract_expires,market_value,player_page
0,1,BRAZIL,1,brasileiro_serie_a,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,3.99,...,Goalkeeper,34,Brazil,1.87,left,2017-07-16,Valencia CF,2020-12-31,3.05,https://www.transfermarkt.com/diego-alves/prof...
1,1,BRAZIL,1,brasileiro_serie_a,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,3.99,...,Goalkeeper,21,Brazil,1.96,both,2019-08-31,Clube de Regatas do Flamengo U20,2023-09-30,0.49,https://www.transfermarkt.com/hugo-souza/profi...
2,1,BRAZIL,1,brasileiro_serie_a,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,3.99,...,Goalkeeper,28,Brazil,1.94,right,2013-01-01,Clube de Regatas do Flamengo U20,2022-04-30,0.44,https://www.transfermarkt.com/cesar/profil/spi...
3,1,BRAZIL,1,brasileiro_serie_a,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,3.99,...,Goalkeeper,21,Brazil,1.88,right,2017-01-01,Clube de Regatas do Flamengo U20,2022-12-31,0.05,https://www.transfermarkt.com/gabriel-batista/...
4,1,BRAZIL,1,brasileiro_serie_a,https://www.transfermarkt.com/campeonato-brasi...,1,Clube de Regatas do Flamengo,33,3,3.99,...,Centre-Back,26,Brazil,1.83,right,2019-01-01,São Paulo Futebol Clube,2023-12-31,5.23,https://www.transfermarkt.com/rodrigo-caio/pro...


In [9]:
pd.read_sql_query('select * from teams', con=engine).head()

,team_id,league_id,club,squad,foreigners,avg_market_value_m,total_mv_m,logo_img,link_page
0,1,1,Clube de Regatas do Flamengo,33,3,3.99,131.52,https://tmssl.akamaized.net/images/wappen/head...,https://www.transfermarkt.com/flamengo-rio-de-...
1,2,1,Sociedade Esportiva Palmeiras,31,3,3.52,109.03,https://tmssl.akamaized.net/images/wappen/head...,https://www.transfermarkt.com/se-palmeiras-sao...
2,3,1,Grêmio Foot-Ball Porto Alegrense,37,2,2.87,106.25,https://tmssl.akamaized.net/images/wappen/head...,https://www.transfermarkt.com/gremio-foot-ball...
3,4,1,São Paulo Futebol Clube,37,5,2.21,81.95,https://tmssl.akamaized.net/images/wappen/head...,https://www.transfermarkt.com/fc-sao-paulo/kad...
4,5,1,Sport Club Corinthians Paulista,33,5,2.44,80.51,https://tmssl.akamaized.net/images/wappen/head...,https://www.transfermarkt.com/corinthians-sao-...


In [15]:
pd.read_sql_query('select * from players where Height > 1.9', con=engine).head().to_json()

'{"players_id":{"0":2,"1":3,"2":7,"3":36,"4":66},"team_id":{"0":1,"1":1,"2":1,"3":2,"4":3},"name":{"0":"Hugo Souza","1":"C\\u00e9sar","2":"Gustavo Henrique","3":"Vinicius Silvestre","4":"J\\u00falio C\\u00e9sar"},"position":{"0":"Goalkeeper","1":"Goalkeeper","2":"Centre-Back","3":"Goalkeeper","4":"Goalkeeper"},"age":{"0":21,"1":28,"2":27,"3":26,"4":33},"nat":{"0":"Brazil","1":"Brazil","2":"Brazil","3":"Brazil","4":"Brazil"},"height":{"0":1.96,"1":1.94,"2":1.96,"3":1.95,"4":1.91},"foot":{"0":"both","1":"right","2":"right","3":"right","4":"right"},"dt_joined":{"0":1567209600000,"1":1356998400000,"2":1578873600000,"3":1372636800000,"4":1546473600000},"prev_team":{"0":"Clube de Regatas do Flamengo U20","1":"Clube de Regatas do Flamengo U20","2":"Santos FC","3":"SE Palmeiras B","4":"Fluminense Football Club"},"contract_expires":{"0":1696032000000,"1":1651276800000,"2":1703980800000,"3":1640908800000,"4":1609372800000},"market_value":{"0":0.49,"1":0.44,"2":2.62,"3":0.11,"4":0.87},"player_pag

In [16]:
team = 'Santos FC '

pd.read_sql_query(\
f"select players.name, players.market_value, teams.club from teams, players where (teams.team_id = players.team_id and teams.club = '{team}')"\
                  , con=engine).to_json()


'{"name":{"0":"\\u00c9verson","1":"Vladimir","2":"John","3":"Jo\\u00e3o Paulo","4":"Lucas Ver\\u00edssimo","5":"Luan Peres","6":"Luiz Felipe","7":"Wagner Leonardo","8":"Felipe Jonatan","9":"Rom\\u00e1rio","10":"Madson","11":"Par\\u00e1","12":"Daniel Guedes","13":"Cadu","14":"Diego Pituca","15":"Alison","16":"Jobson","17":"Jean Mota","18":"Sandry","19":"Ivonei","20":"Carlos S\\u00e1nchez","21":"Evandro","22":"Bryan Ru\\u00edz","23":"Tailson","24":"Alexandre Tam","25":"Yeferson Soteldo","26":"Arthur Gomes","27":"Allanzinho","28":"Eduardo Sasha","29":"Marinho","30":"Lucas Venuto","31":"Kaio Jorge","32":"Yuri Alberto","33":"Raniel","34":"Fernando Uribe"},"market_value":{"0":1.31,"1":0.71,"2":0.05,"3":0.05,"4":3.82,"5":0.6,"6":0.44,"7":0.05,"8":1.42,"9":0.71,"10":0.98,"11":0.71,"12":0.6,"13":0.05,"14":1.96,"15":1.64,"16":0.44,"17":1.74,"18":0.98,"19":0.05,"20":2.18,"21":1.09,"22":0.87,"23":0.74,"24":0.05,"25":8.72,"26":1.42,"27":0.05,"28":4.91,"29":1.42,"30":0.52,"31":7.85,"32":2.51,"33":2.

In [ ]:
# # Create Leagues and Teams Classes
# # ----------------------------------

# # Sets an object to utilize the default declarative base in SQL Alchemy
# Base = declarative_base()


# # Creates Classes which will serve as the anchor points for our Tables
# class Leagues(Base):
#     __tablename__ = 'dog'
#     id = Column(Integer, primary_key=True)
#     name = Column(String(255))
#     color = Column(String(255))
#     age = Column(Integer)


# class Cat(Base):
#     __tablename__ = 'cat'
#     id = Column(Integer, primary_key=True)
#     name = Column(String(255))
#     color = Column(String(255))
#     age = Column(Integer)

In [ ]:
# Leagues
# -
# league_ID Interger PK
# country  String(255)
# tier    Interger
# league_name String(255)
# league_link


# Teams
# -
# team_ID Integer PK
# league_ID Integer FK >- Leagues.league_ID
# club String(255)
# squad Integer
# foreigners Integer
# avg_market_value_m Float
# total_MV_m Float
# Logo_img String(255)
# link_page String(255)

# Players
# -
# players_ID            Integer PK
# team_ID               Integer FK >- Teams.team_ID
# name                 String(255)
# position             String(255)
# Age                   Integer
# Nat                  String(255)
# Height              Float
# foot                 String(255)
# dt_joined            String(255)
# prev_team            String(255)
# contract_expires     String(255)
# market_value        Float
# player_page          String(255)

# Player_status
# -
# players_ID  Integer FK >- Players.players_ID
# league_ID   Interger FK >- Leagues.league_ID
# appearances Interger 
# Goals   Interger 
# assists   Interger
# clean_sheats   Interger
# goals_conceded   Interger
# minutes_played  Interger 

In [ ]:
import sqlalchemy
import pandas as pd
from os.path  import basename
from glob import glob

TABLES = {}
TABLES[]

# MySQL database connection
engine_stmt = 'mysql+mysqldb://%s:%s@%s:3306/%s' % (username, password,
                                                    server,database)
engine = sqlalchemy.create_engine(engine_stmt)

In [ ]:
# get your data into pandas
Leagues_file = glob('data/*.csv', recursive=True)
for filename in files:
    print(filename) 
    df = pd.read_csv(filename)
            for index, row in df_league.iterrows():
            team_id = row["team_ID"]
            team = row["link_page"]
            team_name = team.split('/')[-8]
            # Creates Classes which will serve as the anchor points for our Tables
            class Leagues(Base):
                __tablename__ = filename.split("_")[2]
                team_ID=Column(Integer)
                league_ID=Column(Integer)
                club=Column(String(255))
                squad=Column(Integer)
                foreigners=Column(Integer)
                avg_market_value_m=Column(Float)
                total_MV_m=Column(Float)
                Logo_img=Column(String(255))
                link_page=Column(String(255))
                

In [ ]:
# adjust your dataframe as you want it to look in the database
# df = df.rename(columns={0: 'yearquarter', 1: 'sms_volumes')
# using your existing function to assign start/end row by row
for index, row in df.iterrows():
    dt_start, dt_end = getdatesfromquarteryear(row['yearquarter'])
    df.loc[index, 'sms_start_date'] = dt_start
    df.loc[index, 'sms_end_date'] = dt_end

# write the entire dataframe to database
df.to_sql(name='sms_volumes', con=engine,
          if_exists='append', index=False, chunksize=1000)
print('All data inserted!')